# Can I walk to the train?
In this notebook we'll be exploring the walkability of Klang Valley's train stations using isochrone maps
We will use Open Route Service's API to draw this and compile the results into a csv
The data gathered will be visualized to be understood
KL has multiple different train lines with varying levels of service spreading across a large area. There is a relatively large number of stations relative to the population however the accesibility of the stations seems to be an issue for pedestrians. We'll analyze each train station from different lines. And compile them later on to get an average to be compared with other cities

The work from this notebook was presented in a Twitter thread at the following link:- <br>
https://twitter.com/NabilErsyad/status/1325415849809727489?s=20

In [2]:
#folium is the library used to visualize maps
import folium
#library to call Open Route Service(ORS)'s client and requests
from openrouteservice import client
import pandas as pd 
import numpy as np 
import plotly as py

from datetime import date
from isochrones import * 

#i like to be able to see all my columns in a pandas dataframe. this would allow it 
pd.set_option("display.max_columns", None)


# Train Station isochrones

Let's visualize all the stations and the 15 minute walking radius on a map.
15 minutes is somewhat arbitrary but given malaysia's hot weather it seems the most fair amount of time that malaysian's can generally tolerate to walk outdoors

In [3]:
#Personal api_key stored in a config.py file that is in gitignore. Uncomment the following and provide your own ORS api for your own use
## api_key = "you api key " #Provide your personal API key

from config import api_key

clnt = client.Client(key=api_key) 


In [4]:
#loading up train station data. This file contains the csv data for KL train stations. 
file_url = 'resources/data/raw_data/klang_valley_stations.csv'


In [5]:
#Stop ID is unique for every station. So that column will be used for index
data = pd.read_csv(file_url)
print(data.head())
print(data.columns)

           Name Stop ID Service Provider Name  Latitude   Longitude ROUTE ID  \
0  SENTUL TIMUR     AG1              Rapid KL  3.185821  101.695335       AG   
1          PUDU    AG10              Rapid KL  3.134920  101.712011       AG   
2  CHAN SOW LIN    AG11              Rapid KL  3.128025  101.715546       AG   
3       MIHARJA    AG12              Rapid KL  3.120931  101.717861       AG   
4        MALURI    AG13              Rapid KL  3.123276  101.726946       AG   

    Route Name Line Number Line Colour Colour Hex Code          City  
0  Ampang Line           3      Orange         #fe8e10  Klang Valley  
1  Ampang Line           3      Orange         #fe8e10  Klang Valley  
2  Ampang Line           3      Orange         #fe8e10  Klang Valley  
3  Ampang Line           3      Orange         #fe8e10  Klang Valley  
4  Ampang Line           3      Orange         #fe8e10  Klang Valley  
Index(['Name', 'Stop ID', 'Service Provider Name', 'Latitude', 'Longitude',
       'ROUTE ID'

# isochrone.py package and ORS client to get isochrones
We'll be using the function toMap() that was imported from the custom isochrone.py package. 
All you need for the function are:<br>
-- dataframe with the columns - ['Name','Route Name','Latitude','Longitude'] that would be sufficient to draw isochrone maps for train stations.<br>
-- param_iso - dictionary with the parameters of the isochrone <br>
-- client - the open route service api client <br>

Will return isochrone map of a transit line and dictionary of stations

In [6]:
#the way I coded this is that I want maps that will display train stations by lines. So that it wouldnt clutter the map.
lines = list(data['Route Name'].unique())
maps = []
params_iso = {'profile': 'foot-walking', 
              'range': [900], # 900/60 = 15 mins
              'interval': 300,
              'attributes': ['area', 'reachfactor', 'total_pop'] # Get population count for isochrones
             }
#By creating a list of train lines, i can iterate through each of them and create map objects for each train line.  
# 
# I should edit the toMap function, if params_iso or client is not passed then just map without calling isochrones from ORS           
for line in lines:
    maps.append(toMapORS(data,line,params_iso,clnt))

/home/nabilersyad/Desktop/LinuxDrive/local_repo/train-stations-isochrones/isochrones.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['locations']  = stations.apply(lambda row: list([row.loc["Longitude"],row.loc["Latitude"]]) , axis = 1)


Retrieving Isochrone of SENTUL TIMUR station
Success
Retrieving Isochrone of PUDU station
Success
Retrieving Isochrone of CHAN SOW LIN station
Success
Retrieving Isochrone of MIHARJA station
Success
Retrieving Isochrone of MALURI station
Success
Retrieving Isochrone of PANDAN JAYA station
Success
Retrieving Isochrone of PANDAN INDAH station
Success
Retrieving Isochrone of CEMPAKA station
Success
Retrieving Isochrone of CAHAYA station
Success
Retrieving Isochrone of AMPANG station


/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


Success
Retrieving Isochrone of SENTUL station
Success
Retrieving Isochrone of TITIWANGSA station
Success
Retrieving Isochrone of PWTC station
Success
Retrieving Isochrone of SULTAN ISMAIL station
Success
Retrieving Isochrone of BANDARAYA station
Success
Retrieving Isochrone of MASJID JAMEK station
Success
Retrieving Isochrone of PLAZA RAKYAT station


/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


Success
Retrieving Isochrone of HANG TUAH station
Success
Done!
Retrieving Isochrone of KL SENTRAL station


/home/nabilersyad/Desktop/LinuxDrive/local_repo/train-stations-isochrones/isochrones.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['iso'] = iso_list
/home/nabilersyad/Desktop/LinuxDrive/local_repo/train-stations-isochrones/isochrones.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['locations']  = stations.apply(lambda row: list([row.loc["Longitude"],row.loc["Latitude"]]) , axis = 1)


Success
Retrieving Isochrone of KUALA LUMPUR station
Success
Retrieving Isochrone of BANK NEGARA station
Success
Retrieving Isochrone of PUTRA station
Success
Retrieving Isochrone of MID VALLEY station
Success
Retrieving Isochrone of SEPUTEH station
Success
Retrieving Isochrone of SALAK SELATAN station
Success
Retrieving Isochrone of BANDAR TASIK SELATAN station
Success
Retrieving Isochrone of SERDANG station


/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 5th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 6th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 7th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(ret

Success
Retrieving Isochrone of KAJANG station
Success
Retrieving Isochrone of UKM station
Success
Retrieving Isochrone of BANGI station
Success
Retrieving Isochrone of BATANG BENAR station
Success
Retrieving Isochrone of NILAI station
Success
Retrieving Isochrone of LABU station
Success
Retrieving Isochrone of TIROI station
Success
Retrieving Isochrone of SEREMBAN station
Success
Retrieving Isochrone of SENAWANG station
Success
Retrieving Isochrone of SUNGAI GADUT station
Success
Retrieving Isochrone of REMBAU station
Success
Retrieving Isochrone of PULAU SEBANG/TAMPIN station
Success
Retrieving Isochrone of SENTUL station
Success
Retrieving Isochrone of BATU KENTOMEN station
Success
Retrieving Isochrone of KAMPUNG BATU station
Success
Retrieving Isochrone of TAMAN WAHYU station
Success
Retrieving Isochrone of BATU CAVES station
Success
Done!
Retrieving Isochrone of KL SENTRAL station
Success
Retrieving Isochrone of KUALA LUMPUR station
Success
Retrieving Isochrone of BANK NEGARA stat

# Saving isochrone data into CSVs
Once all the Isochrone requests are completed, I would want to store the data first into CSVs for future references. <br>
The following helper functions from isochrones.py will do that<br>
dictToDataFrame(maps,dataframe)
areaToDataframe(data)

In [17]:
#saves the data with the isochrones  data into new csv files
data= pd.DataFrame(columns=maps[0][1].columns)

for mapz in maps:
    data = data.append(mapz[1])
file_url ='resources/data/klang_valley_stations_isochrones_' + str(date.today())+ '.csv'
data.to_csv(file_url,index=False)

In [18]:
#used to extract more specific data from isochrone data.
data = pd.read_csv(file_url)
data  = areaToDataframe(data)
data.to_csv(file_url,index=False)

In [16]:
maps[0][1]

,Name,Stop ID,Service Provider Name,Latitude,Longitude,ROUTE ID,Route Name,Line Number,Line Colour,Colour Hex Code,City,locations,iso
0,SENTUL TIMUR,AG1,Rapid KL,3.185821,101.695335,AG,Ampang Line,3,Orange,#fe8e10,Klang Valley,"[101.695335, 3.185821]","{""type"": ""FeatureCollection"", ""bbox"": [101.687..."
1,PUDU,AG10,Rapid KL,3.134920,101.712011,AG,Ampang Line,3,Orange,#fe8e10,Klang Valley,"[101.712011, 3.13492]","{""type"": ""FeatureCollection"", ""bbox"": [101.703..."
2,CHAN SOW LIN,AG11,Rapid KL,3.128025,101.715546,AG,Ampang Line,3,Orange,#fe8e10,Klang Valley,"[101.715546, 3.128025]","{""type"": ""FeatureCollection"", ""bbox"": [101.707..."
3,MIHARJA,AG12,Rapid KL,3.120931,101.717861,AG,Ampang Line,3,Orange,#fe8e10,Klang Valley,"[101.717861, 3.120931]","{""type"": ""FeatureCollection"", ""bbox"": [101.710..."
4,MALURI,AG13,Rapid KL,3.123276,101.726946,AG,Ampang Line,3,Orange,#fe8e10,Klang Valley,"[101.726946, 3.1232759999999997]","{""type"": ""FeatureCollection"", ""bbox"": [101.719..."
5,PANDAN JAYA,AG14,Rapid KL,3.130258,101.738991,AG,Ampang Line,3,Orange,#fe8e10,Klang Valley,"[101.738991, 3.130258]","{""type"": ""FeatureCollection"", ""bbox"": [101.731..."
6,PANDAN INDAH,AG15,Rapid KL,3.134827,101.746748,AG,Ampang Line,3,Orange,#fe8e10,Klang Valley,"[101.746748, 3.134827]","{""type"": ""FeatureCollection"", ""bbox"": [101.737..."
7,CEMPAKA,AG16,Rapid KL,3.138458,101.752945,AG,Ampang Line,3,Orange,#fe8e10,Klang Valley,"[101.752945, 3.138458]","{""type"": ""FeatureCollection"", ""bbox"": [101.744..."
8,CAHAYA,AG17,Rapid KL,3.140643,101.756607,AG,Ampang Line,3,Orange,#fe8e10,Klang Valley,"[101.75660699999999, 3.140643]","{""type"": ""FeatureCollection"", ""bbox"": [101.747..."
9,AMPANG,AG18,Rapid KL,3.150512,101.760127,AG,Ampang Line,3,Orange,#fe8e10,Klang Valley,"[101.760127, 3.150512]","{""type"": ""FeatureCollection"", ""bbox"": [101.752..."


In [10]:
maps[0][0]

In [10]:
maps[1][0]

In [11]:
maps[2][0]

In [12]:
maps[3][0]

In [13]:
maps[4][0]

In [14]:
maps[5][0]

In [15]:
maps[6][0]

In [16]:
maps[7][0]

In [17]:
maps[8][0]

In [18]:
maps[9][0]

In [19]:
maps[10][0]

In [11]:
maps[11][0]

# Kuching
Trying out for provisional Kuching

In [5]:
#loading up train station data.
file_url = 'resources/data/raw_data/KUTS_Provisional_Stations.csv'

data_kuch = pd.read_csv(file_url)
lines = list(data_kuch['Route Name'].unique())
map_kuch = []
params_iso = {'profile': 'foot-walking', 
              'range': [900], # 900/60 = 15 mins
              'interval': 300,
              'attributes': ['area', 'reachfactor', 'total_pop'] # Get population count for isochrones
             }
for line in lines:
    map_kuch.append(toMapORS(data_kuch,line,params_iso,clnt))

Retrieving Isochrone of UNIMAS Pintu Timur station
Success
Retrieving Isochrone of Aiman Mall station
Success
Retrieving Isochrone of Wisma SALCRA station
Success
Retrieving Isochrone of UNIMAS station
Success
Retrieving Isochrone of Sarawak Heart Centre station
Success
Retrieving Isochrone of Riveria station
Success
Retrieving Isochrone of NorthBank station
Success
Retrieving Isochrone of Tabuan Jaya Police Complex station
Success
Retrieving Isochrone of VivaCity Megamall station
Success
Retrieving Isochrone of Wisma Persekutuan Public Carpark station
Success
Retrieving Isochrone of The Spring station
Success
Retrieving Isochrone of Kuching Public Works Department station
Success
Retrieving Isochrone of Sarawak General Hospital station
Success
Retrieving Isochrone of Padang Pasir, Hikmah Exchange station
Success
Done!
Retrieving Isochrone of JPJ station


/home/nabilersyad/Desktop/LinuxDrive/local_repo/train-stations-isochrones/isochrones.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['iso'] = iso_list
/home/nabilersyad/Desktop/LinuxDrive/local_repo/train-stations-isochrones/isochrones.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['locations']  = stations.apply(lambda row: list([row.loc["Longitude"],row.loc["Latitude"]]) , axis = 1)


Success
Retrieving Isochrone of 10th Mile Bazaar station
Success
Retrieving Isochrone of SMK Wira Penrissen station
Success
Retrieving Isochrone of Sentosa Parade station
Success
Retrieving Isochrone of Kuching Sentral station
Success
Retrieving Isochrone of Kuching International Airport station


/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(ret

Success
Retrieving Isochrone of RH Plaza station
Success
Retrieving Isochrone of Emporium station
Success
Retrieving Isochrone of Swinburne University station
Success
Retrieving Isochrone of Wisma Persekutuan Public Carpark station
Success
Retrieving Isochrone of Chung Hua Middle School No. 3 station
Success
Retrieving Isochrone of Puspakom station
Success
Retrieving Isochrone of Bintawa Industrial Estate station
Success
Retrieving Isochrone of Borneo Convention Centre Kuching station
Success
Done!


In [10]:
#saves the data with the isochrones  data into new csv files
data_kuch =map_kuch[0][1].append(map_kuch[1][1])
file_url ='resources/data/kuching_provisional_stations_isochrones_' + str(date.today())+ '.csv'
data_kuch.to_csv(file_url,index=False)

In [11]:
#used to extract more specific data from isochrone data.
data_kuch = pd.read_csv(file_url)
data_kuch  = areaToDataframe(data_kuch)
data_kuch.to_csv(file_url,index=False)

In [7]:
map_kuch[0][0]

In [9]:
map_kuch[1][0]

# Singapore
Will be doing the same but with Singapore train station data

In [7]:
#loading up train station data.
file_url = 'resources/data/raw_data/mrtsg.csv'
data_sg = pd.read_csv(file_url)
lines = list(data_sg['Route Name'].unique())
map_sg = []
params_iso = {'profile': 'foot-walking', 
              'range': [900], # 900/60 = 15 mins
              'interval': 300,
              'attributes': ['area', 'reachfactor', 'total_pop'] # Get population count for isochrones
             }
for line in lines:
    map_sg.append(toMapORS(data_sg,line,params_iso,clnt))

/home/nabilersyad/Desktop/LinuxDrive/local_repo/train-stations-isochrones/isochrones.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['locations']  = stations.apply(lambda row: list([row.loc["Longitude"],row.loc["Latitude"]]) , axis = 1)


Retrieving Isochrone of BENCOOLEN MRT STATION station
Success
Retrieving Isochrone of PROMENADE MRT STATION station
Success
Retrieving Isochrone of BUGIS MRT STATION station
Success
Retrieving Isochrone of MACPHERSON MRT STATION station
Success
Retrieving Isochrone of JALAN BESAR MRT STATION station
Success
Retrieving Isochrone of BAYFRONT MRT STATION station
Success
Retrieving Isochrone of FORT CANNING MRT STATION station
Success
Retrieving Isochrone of EXPO MRT STATION station
Success
Retrieving Isochrone of DOWNTOWN MRT STATION station
Success
Retrieving Isochrone of CHINATOWN MRT STATION station
Success
Retrieving Isochrone of NEWTON MRT STATION station
Success
Retrieving Isochrone of TAN KAH KEE MRT STATION station
Success
Retrieving Isochrone of HILLVIEW MRT STATION station
Success
Retrieving Isochrone of KING ALBERT PARK MRT STATION station
Success
Retrieving Isochrone of MATTAR MRT STATION station
Success
Retrieving Isochrone of UBI MRT STATION station
Success
Retrieving Isochr

/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(ret

Success
Retrieving Isochrone of ROCHOR MRT STATION station
Success
Retrieving Isochrone of LITTLE INDIA MRT STATION station
Success
Retrieving Isochrone of STEVENS MRT STATION station
Success
Retrieving Isochrone of BUKIT PANJANG MRT STATION station
Success
Retrieving Isochrone of CASHEW MRT STATION station
Success
Retrieving Isochrone of SIXTH AVENUE MRT STATION station
Success
Retrieving Isochrone of UPPER CHANGI MRT STATION station
Success
Retrieving Isochrone of TELOK AYER MRT STATION station
Success
Retrieving Isochrone of TAMPINES EAST MRT STATION station
Success
Retrieving Isochrone of TAMPINES WEST MRT STATION station
Success
Retrieving Isochrone of GEYLANG BAHRU MRT STATION station
Success
Retrieving Isochrone of BENDEMEER MRT STATION station
Success
Done!
Retrieving Isochrone of BUKIT PANJANG LRT STATION station


/home/nabilersyad/Desktop/LinuxDrive/local_repo/train-stations-isochrones/isochrones.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['iso'] = iso_list
/home/nabilersyad/Desktop/LinuxDrive/local_repo/train-stations-isochrones/isochrones.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['locations']  = stations.apply(lambda row: list([row.loc["Longitude"],row.loc["Latitude"]]) , axis = 1)


Success
Retrieving Isochrone of FAJAR LRT STATION station
Success
Retrieving Isochrone of KEAT HONG LRT STATION station
Success
Retrieving Isochrone of SENJA LRT STATION station
Success
Retrieving Isochrone of JELAPANG LRT STATION station
Success
Retrieving Isochrone of PETIR LRT STATION station
Success
Retrieving Isochrone of TECK WHYE LRT STATION station
Success
Retrieving Isochrone of CHOA CHU KANG LRT STATION station
Success
Retrieving Isochrone of SOUTH VIEW LRT STATION station
Success
Retrieving Isochrone of PENDING LRT STATION station
Success
Retrieving Isochrone of BANGKIT LRT STATION station
Success
Retrieving Isochrone of TEN MILE JUNCTION LRT STATION station


/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 8th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 9th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


Success
Retrieving Isochrone of PHOENIX LRT STATION station
Success
Retrieving Isochrone of SEGAR LRT STATION station
Success
Done!
Retrieving Isochrone of OUTRAM PARK MRT STATION station
Success
Retrieving Isochrone of CLARKE QUAY MRT STATION station
Success
Retrieving Isochrone of DHOBY GHAUT MRT STATION station
Success
Retrieving Isochrone of LITTLE INDIA MRT STATION station
Success
Retrieving Isochrone of POTONG PASIR MRT STATION station
Success
Retrieving Isochrone of HOUGANG MRT STATION station
Success
Retrieving Isochrone of BOON KENG MRT STATION station
Success
Retrieving Isochrone of BUANGKOK MRT STATION station
Success
Retrieving Isochrone of HARBOURFRONT MRT STATION station
Success
Retrieving Isochrone of PUNGGOL MRT STATION station
Success
Retrieving Isochrone of SENGKANG MRT STATION station
Success
Retrieving Isochrone of KOVAN MRT STATION station
Success
Retrieving Isochrone of CHINATOWN MRT STATION station
Success
Retrieving Isochrone of WOODLEIGH MRT STATION station
Suc

In [25]:
#saves the data with the isochrones  data into new csv files
data_temp = pd.DataFrame(columns=map_sg[0][1].columns)
for mapz in map_sg:
    data_temp = data_temp.append(mapz[1])
file_url ='resources/data/mrtsg_isochrones_' + str(date.today())+ '.csv'
data_temp.to_csv(file_url,index=False)

In [26]:
#used to extract more specific data from isochrone data.
data_sg = pd.read_csv(file_url)
data_sg  = areaToDataframe(data_sg)
data_sg.to_csv(file_url,index=False)

In [27]:
map_sg[0][0]

In [28]:
map_sg[1][0]

In [141]:
map_sg[2][0]

In [142]:
map_sg[3][0]

In [143]:
map_sg[4][0]

In [144]:
map_sg[5][0]

In [145]:
map_sg[6][0]

In [146]:
map_sg[][0]

In [147]:
map_sg[8][0]

In [154]:
map_sg[9][0]

In [166]:
map_sg[10][0]

# Montreal
Again same methodology but for Montreal

In [29]:
#loading up train station data.
file_name_montreal = 'resources/data/raw_data/montreal_metro.csv'
data_montreal = pd.read_csv(file_name_montreal)
lines_montreal = list(data_montreal['Route Name'].unique())
maps_montreal = []
params_iso = {'profile': 'foot-walking', 
              'range': [900], # 900/60 = 15 mins
              'interval': 300,
              'attributes': ['area', 'reachfactor', 'total_pop'] # Get population count for isochrones
             }
for line in lines_montreal:
    maps_montreal.append(toMapORS(data_montreal,line,params_iso,clnt))



/home/nabilersyad/Desktop/LinuxDrive/local_repo/train-stations-isochrones/isochrones.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['locations']  = stations.apply(lambda row: list([row.loc["Longitude"],row.loc["Latitude"]]) , axis = 1)


Retrieving Isochrone of Angrignon station
Success
Retrieving Isochrone of Monk station
Success
Retrieving Isochrone of Jolicoeur station
Success
Retrieving Isochrone of Verdun station
Success
Retrieving Isochrone of De L'Église station
Success
Retrieving Isochrone of LaSalle station
Success
Retrieving Isochrone of Charlevoix station
Success
Retrieving Isochrone of Lionel-Groulx station
Success
Retrieving Isochrone of Atwater station
Success
Retrieving Isochrone of Guy–Concordia station
Success
Retrieving Isochrone of Peel station
Success
Retrieving Isochrone of McGill station
Success
Retrieving Isochrone of Place-des-Arts station
Success
Retrieving Isochrone of Saint-Laurent station
Success
Retrieving Isochrone of Berri–UQAM station
Success
Retrieving Isochrone of Beaudry station
Success
Retrieving Isochrone of Papineau station
Success
Retrieving Isochrone of Frontenac station
Success
Retrieving Isochrone of Préfontaine station
Success
Retrieving Isochrone of Joliette station
Success
R

/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(ret

Success
Retrieving Isochrone of Viau station
Success
Retrieving Isochrone of Assomption station
Success
Retrieving Isochrone of Cadillac station
Success
Retrieving Isochrone of Langelier station
Success
Retrieving Isochrone of Radisson station
Success
Retrieving Isochrone of Honoré-Beaugrand station
Success
Done!
Retrieving Isochrone of Côte-Vertu station


/home/nabilersyad/Desktop/LinuxDrive/local_repo/train-stations-isochrones/isochrones.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['iso'] = iso_list
/home/nabilersyad/Desktop/LinuxDrive/local_repo/train-stations-isochrones/isochrones.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['locations']  = stations.apply(lambda row: list([row.loc["Longitude"],row.loc["Latitude"]]) , axis = 1)


Success
Retrieving Isochrone of Du Collège station
Success
Retrieving Isochrone of De La Savane station
Success
Retrieving Isochrone of Namur station
Success
Retrieving Isochrone of Plamondon station
Success
Retrieving Isochrone of Côte-Sainte-Catherine station
Success
Retrieving Isochrone of Snowdon station
Success
Retrieving Isochrone of Villa-Maria station
Success
Retrieving Isochrone of Vendôme station
Success
Retrieving Isochrone of Place-Saint-Henri station
Success
Retrieving Isochrone of Lionel-Groulx station
Success
Retrieving Isochrone of Georges-Vanier station
Success
Retrieving Isochrone of Lucien-L'Allier station
Success
Retrieving Isochrone of Bonaventure station
Success
Retrieving Isochrone of Square-Victoria–OACI station
Success
Retrieving Isochrone of Place-d'Armes station
Success
Retrieving Isochrone of Champ-de-Mars station
Success
Retrieving Isochrone of Berri–UQAM station
Success
Retrieving Isochrone of Sherbrooke station
Success
Retrieving Isochrone of Mont-Royal s

In [32]:
#saves the data with the isochrones  data into new csv files
data_temp = pd.DataFrame(columns=maps_montreal[0][1].columns)
for mapz in maps_montreal:
    data_temp = data_temp.append(mapz[1])
file_name_montreal ='resources/data/montreal_metro_isochrones_' + str(date.today())+ '.csv'
data_temp.to_csv(file_name_montreal,index=False)

In [34]:
#used to extract more specific data from isochrone data.
data_montreal = pd.read_csv(file_name_montreal)
data_montreal  = areaToDataframe(data_montreal)
data_montreal.to_csv(file_name_montreal,index=False)

In [35]:
maps_montreal[0][0]

In [260]:
maps_montreal[1][0]

In [261]:
maps_montreal[2][0]

In [262]:
maps_montreal[3][0]

# We've got isochrones!

We're done gathering data. Now to get some stats we'll be doing it in a different notebook as it's getting too long here